Deep Lerning. We will search G1, G2 and after G3.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Concatenate
from tensorflow.keras import regularizers
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split


In [2]:
student_performance = fetch_ucirepo(id=320) 

data_X = student_performance.data.features
data_y = student_performance.data.targets

X = pd.get_dummies(data_X)
y = data_y

# При использовании pd.get_dummies для преобразования категориальных признаков в dummy-переменные, нет необходимости в явном использовании слоя нормализации в модели. 
# Это потому, что pd.get_dummies создает бинарные (или dummy) переменные, которые по своей природе уже нормализованы (их значения находятся в диапазоне [0, 1]).
# слой Flatten так же не нужен


In [3]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
x_train = x_train.astype(float)
x_test = x_test.astype(float)

y_train = y_train.astype(float)
y_test = y_test.astype(float)

In [5]:
input_scores = Input(shape=(x_train.shape[1],))

# Слои для предсказания g1
dense_g1 = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.001))(input_scores)
output_g1 = Dense(1)(dense_g1)

# Слои для предсказания g2 (используя input_scores и g1)
concat_g2 = Concatenate()([input_scores, output_g1])
dense_g2 = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.001))(concat_g2)
output_g2 = Dense(1)(dense_g2)
# Слои для предсказания g3 (используя input_scores, g1 и g2)
concat_g3 = Concatenate()([input_scores, output_g1, output_g2])
dense_g3 = Dense(16, activation="relu", kernel_regularizer=regularizers.l2(0.001))(concat_g3)
output_g3 = Dense(1)(dense_g3)

model = Model(inputs=input_scores, outputs=[output_g1, output_g2, output_g3])

In [6]:
model.compile(optimizer="adam", loss="mse", metrics=["mse", "mse", "mse"])

In [7]:
model.fit(x_train, [y_train['G1'], y_train['G2'], y_train['G3']], epochs=300)

Epoch 1/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - dense_1_mse: 120.4739 - dense_3_mse: 152.0926 - dense_5_mse: 280.8297 - loss: 553.4671
Epoch 2/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_1_mse: 77.7995 - dense_3_mse: 91.7139 - dense_5_mse: 191.2490 - loss: 360.8306  
Epoch 3/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_1_mse: 45.6281 - dense_3_mse: 43.1610 - dense_5_mse: 113.4692 - loss: 202.3250 
Epoch 4/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_1_mse: 30.1064 - dense_3_mse: 19.9045 - dense_5_mse: 57.5114 - loss: 107.5882 
Epoch 5/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_1_mse: 18.6585 - dense_3_mse: 13.8494 - dense_5_mse: 27.2160 - loss: 59.7891 
Epoch 6/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_1_mse: 12.8095 - dense_3_mse: 12.6476 - dense_5_mse: 16.0804 - loss: 41.6017 
Epoch 7/300
17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - dense_1_mse: 11.0278 - dense_3_mse: 12.9295 - dense_5_mse: 15.7890 - loss: 39.8093 
Epoch 8/300
17/17 ━━━━━━━━━━━━━━

In [8]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 56)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 16)        │        912 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 1)         │         17 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 57)        │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ dense_1[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 16)        │        928 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_3 (Dense)     │ (None, 1)         │         17 │ dense_2[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate_1       │ (None, 58)        │          0 │ input_layer[0][0… │
│ (Concatenate)       │                   │            │ dense_1[0][0],    │
│                     │                   │            │ dense_3[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_4 (Dense)     │ (None, 16)        │        944 │ concatenate_1[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_5 (Dense)     │ (None, 1)         │         17 │ dense_4[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 8,507 (33.23 KB)

 Trainable params: 2,835 (11.07 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,672 (22.16 KB)

In [9]:
model.evaluate(x_test, y_test)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - dense_1_mse: 8.7830 - dense_3_mse: 0.0000e+00 - dense_5_mse: 0.0000e+00 - loss: 8.8936  


[9.333623886108398, 9.223052024841309, 0.0, 0.0]

In [10]:
pred = model.predict(x_train)
pred = np.clip(pred, 0, 20).astype(int) # обрезка
pd.DataFrame(pred.reshape(519, 3)) # преобразуем в двухмерный массив

17/17 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step


,0,1,2
0,13,10,12
1,11,7,9
2,6,8,7
3,12,10,12
4,12,11,9
...,...,...,...
514,12,13,14
515,12,10,13
516,8,12,10
517,12,13,12


In [11]:
pd.DataFrame(data_y)

,G1,G2,G3
0,0,11,11
1,9,11,11
2,12,13,12
3,14,14,14
4,11,13,13
...,...,...,...
644,10,11,10
645,15,15,16
646,11,12,9
647,10,10,10
